# VGG-16

Papers [Are You Paying Attention? Detecting Distracted Driving in Real-Time](https://ieeexplore.ieee.org/document/8919430) and [Detection of Distracted Driver using Convolutional Neural Network](https://openaccess.thecvf.com/content_cvpr_2018_workshops/papers/w14/Baheti_Detection_of_Distracted_CVPR_2018_paper.pdf) illustrate how VGG-16 can produce high quality predictions. In this notebook, I utilize VGG-16 and train the final classification layers to see if I can get a good prediction.

In [1]:
# Imports
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2
from torchvision.models import vgg16_bn
from sklearn.metrics import log_loss, accuracy_score

## Classes & Functions

In [2]:
class StateFarmDD(Dataset):
    # Constructor
    def __init__(self,annotations_path,path_prefix='../data/imgs/train',transform_pipeline=None):
        self.annotations = pd.read_csv(annotations_path)
        self.transformation_pipeline = transform_pipeline
        self.path_prefix = path_prefix
        self.label_to_int_dict = {'c0':0,'c1':1,'c2':2,'c3':3,'c4':4,'c5':5,'c6':6,'c7':7,'c8':8,'c9':9}
    
    # Method to get the length of the dataset
    def __len__(self):
        return len(self.annotations)
    
    # Method to get the item at a particular index
    def __getitem__(self,index):
        label = self.annotations.iloc[index,1]
        image_name = self.annotations.iloc[index,2]
        image = plt.imread(f'{self.path_prefix}/{label}/{image_name}')

        # Throwing image through pipeline if it exists
        if self.transformation_pipeline:
            transformed_image = self.transformation_pipeline(image.copy()).squeeze(0)
            return transformed_image, self.label_to_int_dict[label]
        else:
            return image, self.label_to_int_dict[label]

In [3]:
# Function to calculate the metrics
def calculate_metrics(model_id,model_name,training_targets,training_predictions,val_targets,val_predictions):

    # Getting the metrics
    train_CE_loss = log_loss(training_targets,training_predictions)
    validation_CE_loss = log_loss(val_targets,val_predictions)

    # Getting the accuracy
    train_class_preds = np.array(training_predictions).argmax(axis=1)
    valid_class_preds = np.array(val_predictions).argmax(axis=1)
    train_acc = accuracy_score(training_targets,train_class_preds)
    valid_acc = accuracy_score(val_targets,valid_class_preds)

    return {'model_id':model_id,'model_name':model_name,'train_CE_loss':train_CE_loss,
            'train_acc':train_acc,'validation_CE_loss':validation_CE_loss,'validation_acc':valid_acc}

## Model Building & Training

In [4]:
# Getting the data
training_path = '/kaggle/input/statefarmdd/training.csv'
validation_path = '/kaggle/input/statefarmdd/validation.csv'

# Creating the Datasets
transformation_pipeline = v2.Compose([
    v2.Resize(256),
    v2.CenterCrop(224),
    v2.ToImage(), 
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
train_dataset = StateFarmDD(training_path, path_prefix='/kaggle/input/state-farm-distracted-driver-detection/imgs/train',transform_pipeline=transformation_pipeline)
valid_dataset = StateFarmDD(validation_path, path_prefix='/kaggle/input/state-farm-distracted-driver-detection/imgs/train',transform_pipeline=transformation_pipeline)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True)

In [5]:
# Defining the model
model = vgg16_bn(weights='DEFAULT')
for param in model.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True

model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)

# Training the model
model_name = 'vgg16_1'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
loss_fn = torch.nn.CrossEntropyLoss(reduction='sum').to(device)
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)
training_history_loss = []
validation_history_loss = []
train_history_accuracy = []
valid_history_accuracy = []
epochs = 100
current_count = 0
early_stopping_threshold = 1e-4
early_stopping_count = 5
best_val_loss = float('inf')
best_epoch = -1

# Training the model
for epoch in range(epochs):
    model.train()
    train_loss = 0
    valid_loss = 0
    train_accuracy = 0
    valid_accuracy = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Computing the accuracy
        y_pred_labels = torch.argmax(y_pred, dim=1)
        train_accuracy += torch.sum(y_pred_labels == y_batch).item()

    model.eval()
    with torch.no_grad():
        for X_batch, y_batch in valid_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            valid_loss += loss.item()

            # Computing the accuracy
            y_pred_labels = torch.argmax(y_pred, dim=1)
            valid_accuracy += torch.sum(y_pred_labels == y_batch).item()
    
    train_loss /= len(train_loader.dataset)
    valid_loss /= len(valid_loader.dataset)
    train_accuracy /= len(train_loader.dataset)
    valid_accuracy /= len(valid_loader.dataset)
    training_history_loss.append(train_loss)
    validation_history_loss.append(valid_loss)
    train_history_accuracy.append(train_accuracy)
    valid_history_accuracy.append(valid_accuracy)
    
    # Early stopping
    if epoch > 0:
        if validation_history_loss[-1] - best_val_loss > early_stopping_threshold:
            current_count += 1
        else:
            best_val_loss = validation_history_loss[-1]
            best_epoch = epoch
            current_count = 0
    else:
        best_val_loss = validation_history_loss[-1]
        best_epoch = epoch
        
    if current_count == early_stopping_count:
        print('Stopping training due to early stopping!!!')
        break
    elif current_count == 0:
        # Saving the best model
        model.to('cpu')
        torch.save(model.state_dict(), f'{model_name}.pth')
        model.to(device)
    print('-----------------------------------')
    print(f'Epoch {epoch}')
    print(f'Training Loss: {round(train_loss,4)}')
    print(f'Validation Loss: {round(valid_loss,4)}')
    print(f'Training Accuracy: {round(train_accuracy*100,4)}%')
    print(f'Validation Accuracy: {round(valid_accuracy*100,4)}%')
    print()
    print(f'Best Validation Loss: {round(best_val_loss,4)}')
    print(f'Best Epoch: {best_epoch}')
    print('-----------------------------------')
    print()

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:02<00:00, 205MB/s]


-----------------------------------
Epoch 0
Training Loss: 0.8849
Validation Loss: 3.0544
Training Accuracy: 70.0936%
Validation Accuracy: 43.2091%

Best Validation Loss: 3.0544
Best Epoch: 0
-----------------------------------

-----------------------------------
Epoch 1
Training Loss: 0.2394
Validation Loss: 2.2857
Training Accuracy: 94.423%
Validation Accuracy: 53.6262%

Best Validation Loss: 2.2857
Best Epoch: 1
-----------------------------------

-----------------------------------
Epoch 2
Training Loss: 0.2443
Validation Loss: 3.8067
Training Accuracy: 95.0717%
Validation Accuracy: 50.9612%

Best Validation Loss: 2.2857
Best Epoch: 1
-----------------------------------

-----------------------------------
Epoch 3
Training Loss: 0.3102
Validation Loss: 3.5306
Training Accuracy: 95.1341%
Validation Accuracy: 50.5864%

Best Validation Loss: 2.2857
Best Epoch: 1
-----------------------------------

-----------------------------------
Epoch 4
Training Loss: 0.2849
Validation Loss: 3.

In [6]:
# Saving Plots for the training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(training_history_loss, label='Training Loss')
plt.plot(validation_history_loss, label='Validation Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.savefig(f'{model_name}_loss.png')
plt.close()

# Saving plot for the training and validation accuracy 
plt.figure(figsize=(10, 6))
plt.plot(train_history_accuracy, label='Training Accuracy')
plt.plot(valid_history_accuracy, label='Validation Accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.savefig(f'{model_name}_accuracy.png')
plt.close()

In [7]:
# Making the predictions for the training & validation for metric logging
model.load_state_dict(torch.load(f'{model_name}.pth')) # loading the best model
model.eval()
model.to(device)
train_pred = []
valid_pred = []
train_truth = []
valid_truth = []

/tmp/ipykernel_23/1877011205.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'{model_name}.pth')) # loading the best model


In [8]:
# Running through data loaders to store the predictions
with torch.no_grad():
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_pred = torch.nn.functional.softmax(model(X_batch),dim=1)
        train_pred.extend(y_pred.detach().cpu().numpy())
        train_truth.extend(y_batch.numpy())

    for X_batch, y_batch in valid_loader:
        X_batch = X_batch.to(device)
        y_pred = torch.nn.functional.softmax(model(X_batch),dim=1)
        valid_pred.extend(y_pred.detach().cpu().numpy())
        valid_truth.extend(y_batch.numpy())

# Printing out the metrics 
metrics = calculate_metrics('15',f'{model_name}',train_truth,np.array(train_pred),valid_truth,np.array(valid_pred))
print(metrics)

{'model_id': '15', 'model_name': 'vgg16_1', 'train_CE_loss': 0.14113247499081977, 'train_acc': 0.9799126637554585, 'validation_CE_loss': 2.2852636671903293, 'validation_acc': 0.5362620584357}
